SOME BUGS PLS USE THE SELENIUM VERSION


In [22]:
from playwright.async_api import async_playwright
import asyncio

In [23]:
"""
<td>
    <div class="obc-tmpl-character__voice-item obc-tmpl-character__play-voice">
        <div class="obc-tmpl-character__voice-btn">
        </div> 
        <span class="obc-tmpl-character__voice-content"> 	
            [text] 	 
        </span>
    </div>
 </td>
"""

'\n<td>\n    <div class="obc-tmpl-character__voice-item obc-tmpl-character__play-voice">\n        <div class="obc-tmpl-character__voice-btn">\n        </div> \n        <span class="obc-tmpl-character__voice-content"> \t\n            [text] \t \n        </span>\n    </div>\n </td>\n'

In [24]:
URL = "https://bbs.mihoyo.com/ys/obc/content/503614/detail"
TRAGET_FEATURE="https://act-upload.mihoyo.com/wiki-user-upload"
OUTPUT_DIR = "/tmp/audio"

In [26]:
async def scrape_texts_and_urls():
    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context() 
        page = await context.new_page()
        
        # Intercept all requests and add a "Cache-Control: no-cache" header.
        await page.route("**/*", lambda route, request: route.continue_(
            headers={**request.headers, "Cache-Control": "no-cache"}
        ))
        
        # Navigate to the page and wait for the containers.
        await page.goto(URL)
        await page.wait_for_selector("div.obc-tmpl-character__voice-item")
        
        # Select all containers that include both the text and the voice button.
        containers = await page.query_selector_all("div.obc-tmpl-character__voice-item.obc-tmpl-character__play-voice")
        
        for index, container in enumerate(containers):
            # Extract the text.
            text_el = await container.query_selector("span.obc-tmpl-character__voice-content")
            text = (await text_el.text_content()).strip() if text_el else ""
            
            # Locate the voice button.
            button = await container.query_selector("div.obc-tmpl-character__voice-btn")
            if not button:
                print(f"Button not found in container {index}")
                results.append((text, None))
                continue
            
            # Ensure the container is visible.
            await container.scroll_into_view_if_needed()
            
            mp3_url = None
            while mp3_url is None:
                try:
                    # Wait for the .mp3 GET request triggered by clicking the button.
                    async with page.expect_request(
                        lambda request: request.method == "GET" and request.url.endswith(".mp3"),
                        timeout=5000
                    ) as request_info:
                        await button.scroll_into_view_if_needed()
                        await button.click(
                            timeout=8000,
                            force=True,
                            delay=100,
                            trial=True  
                        )
                        await button.evaluate("element => { element.style.pointerEvents = 'auto'; element.click() }")
                        # Brief pause to allow the request to be initiated.
                        await asyncio.sleep(0.5)
                    mp3_request = await request_info.value
                    mp3_url = mp3_request.url
                    print(f"Captured mp3 for container {index}: {mp3_url}")
                except Exception as e:
                    #print(f"Retrying for container {index} text '{text}' due to: {e}")
                    #await asyncio.sleep(1)
                    mp3_url=""
            
            results.append((text, mp3_url))
        
        await browser.close()
    return results

l=await scrape_texts_and_urls()
    


Captured mp3 for container 0: https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/5c5b0c7d4f18fdd601fa684ac3f2dddf_1601913573539800143.mp3
Captured mp3 for container 1: https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/98660f0a7e0d7a0f5ecf9a5ebce8080e_4716642393252047127.mp3
Captured mp3 for container 2: https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/b704c1e44aaf1462a91a5a1b03166fae_3657252695928088414.mp3
Captured mp3 for container 3: https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/516d8bd61233672cb706058ee93489d8_525339030534888011.mp3
Captured mp3 for container 4: https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/efbcb36835fcd63dd181191ba69a73bb_6685150892736786968.mp3
Captured mp3 for container 5: https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/77d7683573809d10db98d42261272401_4467602525331314027.mp3
Captured mp3 for container 6: https://act-upload.mihoyo.com/wiki-user-upload/

CancelledError: 

In [5]:
l[:5]

[('你好，我是沉玉谷手工艺工会的蓝砚。是想做点摆件、篮子还是花瓶、家具之类的呢？呃，备用的藤枝藤条不多了，得先去山里采一趟…你也想帮忙？嘻嘻，你人真好，我多送你一顶藤编遮阳帽可好？',
  'https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/5c5b0c7d4f18fdd601fa684ac3f2dddf_1601913573539800143.mp3'),
 ('今天的计划？嗯…听随当下的心意而动。明天也是，每天都是，嘻嘻。',
  'https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/98660f0a7e0d7a0f5ecf9a5ebce8080e_4716642393252047127.mp3'),
 ('好的藤器经篾薄如绸，纬条细若丝，经久不坏…说它能跟银器一样当传家宝一点都不夸张。',
  'https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/b704c1e44aaf1462a91a5a1b03166fae_3657252695928088414.mp3'),
 ('天暖燕子么搭窝窝，升雷扯闪么我家躲。',
  'https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/516d8bd61233672cb706058ee93489d8_525339030534888011.mp3'),
 ('赶不及躲雨了，那——安心淋雨吧，还省了跑步的力气。',
  'https://act-upload.mihoyo.com/wiki-user-upload/2025/01/24/16576950/efbcb36835fcd63dd181191ba69a73bb_6685150892736786968.mp3')]

In [6]:
import os
import requests

In [20]:
quarter = len(l) // 4
marks=["C","J","E","K"]

In [9]:
# Headers from network inspection
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0",
    "Referer": "https://bbs.mihoyo.com/",
    "Accept": "audio/webm,audio/ogg,audio/wav,audio/*;q=0.9,application/ogg;q=0.7,video/*;q=0.6,*/*;q=0.5",
    "DNT": "1",
    "Sec-Fetch-Dest": "audio",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "same-site"
}

In [16]:
person_id=URL.split('/')[-2]
person_id

'4073'

In [12]:
for i, (text, url) in enumerate(l):
    if url:
        q = i // quarter
        response = requests.get(url, headers=HEADERS, stream=True)
        response.raise_for_status()
        dir=os.path.join(OUTPUT_DIR,person_id,marks[q])
        os.makedirs(dir, exist_ok=True)
        with open(os.path.join(dir,f"{i-q*quarter}.mp3"),"wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:  # Filter out keep-alive chunks
                    f.write(chunk)



In [21]:
dir=os.path.join(OUTPUT_DIR,person_id)
os.makedirs(dir, exist_ok=True)
with open(os.path.join(dir,"index.csv"),"w") as f:
    i=0
    while i<quarter:
        f.write(f"{i},{l[i][0]}\n")
        i+=1
